In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
from PIL import Image
import math

# Following code was used to create dataframe for heatmap

In [32]:
inputdir="C:\\Users\\sunad\\Desktop\\exp210919_cantas\\tissue_segmentation_220305\\segmentation\\96h\\for_graph"


df_t1 = pd.read_csv(f"{inputdir}\\output\\tissue1_amplicon_density_roi.csv")
df_t1 = df_t1.rename(columns={'Unnamed: 0': 'gene'})

df_t2 = pd.read_csv(f"{inputdir}\\output\\tissue2_amplicon_density_roi.csv")
df_t2 = df_t2.rename(columns={'Unnamed: 0': 'gene'})

df_t3 = pd.read_csv(f"{inputdir}\\output\\tissue3_amplicon_density_roi.csv")
df_t3 = df_t3.rename(columns={'Unnamed: 0': 'gene'})

genelist = pd.read_csv(f"{inputdir}\\genelist_all.csv")
x_length = range(-180, 430, 30)
x_axis = list(x_length)

"""
Minimum and maximum of x-axis is set to make graph.
For each-tissue graph, max-x and min-x were set to cover the entire tissue.
For average of three tissue graph, max-x and min-x were set to the shortest length

for all: 96h: x_length = range(-180, 430, ...)
for all: 120h: x_length = range(-140, 980, ...)

for each: 120h_tissue1: x_length(-180, 980, 40)
for each: 120h_tissue3: x_length(-140, 1000, 40)
for each: 120h_tissue4: x_length(-140, 1720, 40)

for each: 96h_tissue1: x_length(-260, 430, 30)
for each: 96h_tissue2: x_length(-180, 510, 30)
for each: 96h_tissue3: x_length(-220, 470, 30)
"""


'\nMinimum and maximum of x-axis is set to make graph.\nFor each-tissue graph, max-x and min-x were set to cover the entire tissue.\nFor average of three tissue graph, max-x and min-x were set to the shortest length\n\nfor all: 96h: x_length = range(-180, 430, ...)\nfor all: 120h: x_length = range(-140, 980, ...)\n\nfor each: 120h_tissue1: x_length(-180, 980, 40)\nfor each: 120h_tissue3: x_length(-140, 1000, 40)\nfor each: 120h_tissue4: x_length(-140, 1720, 40)\n\nfor each: 96h_tissue1: x_length(-260, 430, 30)\nfor each: 96h_tissue2: x_length(-180, 510, 30)\nfor each: 96h_tissue3: x_length(-220, 470, 30)\n'

In [12]:
"""
here, density of amplicon is calculated according to x_length = range(.., .., ..)
# data of (center of mass of segment, gene spot density) was connected with line,
and density of gene spot to list of x_length was calculated

"""
def density_plotter(df, genename):
    
    amp_density_list = []
    a = df.loc[df.gene == "distance_micro-m", :].values[0].tolist()
    del a[0]
    g = df.loc[df.gene == genename, :].values[0].tolist()
    del g[0]

    for i in x_length:
        b = [(x-i)**2 for x in a]
        c = sorted(enumerate(b), key = lambda x: x[1])
        idx1 = c[0][0]
        idx2 = c[1][0]
        
        alpha = (g[idx1]-g[idx2])/(a[idx1]-a[idx2])
            
        m = g[idx1] - alpha*(a[idx1]-i)
            
        amp_density_list.append(m)

        
        
    return amp_density_list
    
    

In [27]:
#making dataframe for heatmap_average
outputdir = f"{inputdir}\\output"
df_average = pd.DataFrame(columns = x_axis)

for i in range(len(genelist)):
    genename = genelist.loc[i, "Gene"]
    
    amp_list1 = density_plotter(df_t1, genename)
    amp_list2 = density_plotter(df_t2, genename)
    amp_list3 = density_plotter(df_t3, genename)
    
    amp_list_ave = []
    
    for k in range(len(amp_list1)):
        s = (amp_list1[k] + amp_list2[k] + amp_list3[k])/3
        amp_list_ave.append(s)
    
    df_average.loc[genename] = amp_list_ave
        
    
df_average.to_csv(f"{outputdir}\\average_heatmap_amplicon_density_roi.csv")
    



In [25]:
#making dataframe for heatmap_each_tissue

outputdir = f"{inputdir}\\output"
df = pd.DataFrame(columns = x_axis)

for i in range(len(genelist)):
    genename = genelist.loc[i, "Gene"]
    
    amp_list = density_plotter(df_t3, genename)
        
    df.loc[genename] = amp_list
        
    
df.to_csv(f"{outputdir}\\tissue3_heatmap_amplicon_density_roi.csv")